In [6]:
#Input from sensor
input_analog=250 #Analog value are obtained from sensor (linear potentio) 
km=20000 #Mileage are obtained from odometry in km

#Brakepad thickness calculation
d_a=1 #Sensor initial condition in mm
d_max=26 #Maximum sensor displacement in mm
bit=1023 #Analog to digital converter 10 bit
d_bpmax=20 #Maximum brakepad thickness in mm
d_gapin=5 #initial gap between brake pad and disc in mm

d_sen=(input_analog/bit)*d_max #Sensor reading value in mm
#print (d_sen)
d_gap=d_sen-d_a #Recent gap between brake pad and disc in mm
#print (d_gap)
d_bp=d_bpmax-(d_gap-d_gapin) #Result brakepad thickness in mm
formatted_d_bp = "{:.2f}".format(d_bp)
print ("Brakepad thickness:",formatted_d_bp, "mm")

#Brakepad percentage
percentage=(d_bp/d_bpmax)*100
percentage=int(percentage)
print ("Brakepad: ", percentage, "%")



Brakepad thickness: 19.65 mm
Brakepad:  98 %


In [7]:
#Decision-Tree-for making warning
from sklearn import tree
from graphviz import Digraph
clf = tree.DecisionTreeClassifier() 

#[thickness-mm, percentage-%, mileage]                                             
X = [ [20, 100, 1440],                                                              
      [16, 80, 9870],                                                              
      [12, 60, 17720],                                                              
      [4, 20, 29106],                                                              
      [2, 10, 37910]]                                                              

Y = ['Mint', 'Very Good', 'Caution', 'Danger', 'Replace']

clf = clf.fit(X, Y)                                                             
prediction = clf.predict([[d_bp, percentage, km]])                                         
print(prediction) 
tree.export_graphviz(clf,
    out_file='tree1.dot')     

import graphviz
from sklearn.tree import export_graphviz
graphviz.Source.from_file('tree1.dot')
! dot -Tpng tree1.dot -o tree.png 

prediction=str(prediction)
prediction_bersih1=prediction.replace("[","")
prediction_bersih2=prediction_bersih1.replace("]","")
prediction_bersih3=prediction_bersih2.replace("'","")

['Caution']


In [3]:
#Import dataset
import pandas as pd

excel_file=r'.\dataset\need_maintenance.xlsx'
df = pd.read_excel (excel_file, index_col=0)
df.head()

,battery,brakepad,motor
0,FAIL,NORMAL,FAIL


In [8]:
#Edit brakepad conditon in dataset
if prediction_bersih3=='MINT' or prediction_bersih3=='Very Good':
    if df.iloc[0][1]=='FAIL':
        df['breakpad'] = df['breakpad'].replace(['FAIL'],'NORMAL')
elif prediction_bersih3=='Danger' or  prediction_bersih3=='Caution' or prediction_bersih3=='Replace':
    if df.iloc[0][1]=='NORMAL':
        df['breakpad'] = df['breakpad'].replace(['NORMAL'],'FAIL')

df.to_excel(excel_file)
print(df.head())

  battery brakepad motor
0    FAIL   NORMAL  FAIL
